In [2]:
print("hello world!!!!")

hello world!!!!


In [3]:
import os, json
from sentence_transformers import SentenceTransformer
from langchain_core.vectorstores import InMemoryVectorStore
import numpy as np
from collections import defaultdict

C:\Users\LNPQ8256\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# creating patient database list
patient_detail_path = "Patient_Details"
patient_list = []
for patient_file in os.listdir(patient_detail_path):
    per_patient_detail = {}
    file = open(os.path.join(patient_detail_path, patient_file), "r")
    for line in file.readlines():
        if line.startswith("Patient_ID"):
            per_patient_detail["patient_id"] = line.split(":")[1].strip()
        if line.startswith("Age"):
            per_patient_detail["age"] = line.split(":")[1].strip()
        if line.startswith("Gender"):
            per_patient_detail["gender"] = line.split(":")[1].strip()
    file = open(os.path.join(patient_detail_path, patient_file), "r")
    for section in file.read().split("--- "):
        if section.startswith("Symptoms"):
            per_patient_detail["symptoms"] = section.split("---")[1].strip().replace("\n", ", ")
        if section.startswith("Vitals"):
            per_patient_detail["vitals"] = section.split("---")[1].strip().replace("\n", ", ")
        if section.startswith("Lab Reports"):
            per_patient_detail["lab_reports"] = section.split("---")[1].strip().replace("\n",", ")
        if section.startswith("Doctor Notes"):
            per_patient_detail["doctor_notes"] = section.split("---")[1].strip().replace("\n", " ")
        if section.startswith("Final Department"):
            per_patient_detail["final_department"] = section.split("---")[1].strip().replace("\n", " ")
        if section.startswith("Outcome"):
            per_patient_detail["outcome"] = section.split("---")[1].strip().replace("\n", " ")
    patient_list.append(per_patient_detail)

print(patient_list)
print(patient_list[0])

[{'patient_id': 'P_10001', 'age': '45', 'gender': 'Male', 'symptoms': 'Chest tightness for 2 days, Mild shortness of breath, Sweating', 'vitals': 'BP: 140/90, HR: 92', 'lab_reports': 'ECG: Normal, Troponin: Normal', 'doctor_notes': 'Cardiac evaluation advised', 'final_department': 'Cardiology', 'outcome': 'OPD evaluation done'}, {'patient_id': 'P_10010', 'age': '55', 'gender': 'Male', 'symptoms': 'Chest pain radiating to left arm, Shortness of breath, Nausea', 'vitals': 'BP: 170/105, HR: 110', 'lab_reports': 'ECG: ST elevation, Troponin: Elevated', 'doctor_notes': 'Acute myocardial infarction suspected', 'final_department': 'Cardiology', 'outcome': 'Emergency angioplasty done'}, {'patient_id': 'P_10011', 'age': '36', 'gender': 'Female', 'symptoms': 'Neck pain, Stiffness, Reduced range of motion', 'vitals': 'BP: 118/76, HR: 74', 'lab_reports': 'X-ray Cervical Spine: Muscle spasm', 'doctor_notes': 'Cervical spondylosis suspected', 'final_department': 'Orthopedics', 'outcome': 'Physiother

In [5]:
patient_list[0]

{'patient_id': 'P_10001',
 'age': '45',
 'gender': 'Male',
 'symptoms': 'Chest tightness for 2 days, Mild shortness of breath, Sweating',
 'vitals': 'BP: 140/90, HR: 92',
 'lab_reports': 'ECG: Normal, Troponin: Normal',
 'doctor_notes': 'Cardiac evaluation advised',
 'final_department': 'Cardiology',
 'outcome': 'OPD evaluation done'}

In [6]:
# creating chunks
chunks = []
for patient in patient_list:
    chunks.append({
        "text": f"Symptoms: {patient['symptoms']}",
        "metadata": {
            "patient_id": patient["patient_id"],
            "chunk_type": "symptoms",
            "department": patient["final_department"]
        }
        })
    chunks.append({
        "text": f"Vitals: {patient['vitals']}\nLab results: {patient['lab_reports']}",
        "metadata": {
            "patient_id": patient["patient_id"],
            "chunk_type": "clinical",
            "department": patient["final_department"]
        }
        })
    chunks.append({
        "text": f"Doctor Assesment: {patient['doctor_notes']}",
        "metadata": {
            "patient_id": patient["patient_id"],
            "chunk_type": "assessment",
            "department": patient["final_department"]
        }
        })

print(chunks)

[{'text': 'Symptoms: Chest tightness for 2 days, Mild shortness of breath, Sweating', 'metadata': {'patient_id': 'P_10001', 'chunk_type': 'symptoms', 'department': 'Cardiology'}}, {'text': 'Vitals: BP: 140/90, HR: 92\nLab results: ECG: Normal, Troponin: Normal', 'metadata': {'patient_id': 'P_10001', 'chunk_type': 'clinical', 'department': 'Cardiology'}}, {'text': 'Doctor Assesment: Cardiac evaluation advised', 'metadata': {'patient_id': 'P_10001', 'chunk_type': 'assessment', 'department': 'Cardiology'}}, {'text': 'Symptoms: Chest pain radiating to left arm, Shortness of breath, Nausea', 'metadata': {'patient_id': 'P_10010', 'chunk_type': 'symptoms', 'department': 'Cardiology'}}, {'text': 'Vitals: BP: 170/105, HR: 110\nLab results: ECG: ST elevation, Troponin: Elevated', 'metadata': {'patient_id': 'P_10010', 'chunk_type': 'clinical', 'department': 'Cardiology'}}, {'text': 'Doctor Assesment: Acute myocardial infarction suspected', 'metadata': {'patient_id': 'P_10010', 'chunk_type': 'asse

In [7]:
chunks[0:4]

[{'text': 'Symptoms: Chest tightness for 2 days, Mild shortness of breath, Sweating',
  'metadata': {'patient_id': 'P_10001',
   'chunk_type': 'symptoms',
   'department': 'Cardiology'}},
 {'text': 'Vitals: BP: 140/90, HR: 92\nLab results: ECG: Normal, Troponin: Normal',
  'metadata': {'patient_id': 'P_10001',
   'chunk_type': 'clinical',
   'department': 'Cardiology'}},
 {'text': 'Doctor Assesment: Cardiac evaluation advised',
  'metadata': {'patient_id': 'P_10001',
   'chunk_type': 'assessment',
   'department': 'Cardiology'}},
 {'text': 'Symptoms: Chest pain radiating to left arm, Shortness of breath, Nausea',
  'metadata': {'patient_id': 'P_10010',
   'chunk_type': 'symptoms',
   'department': 'Cardiology'}}]

In [8]:
# creating embedding
embeddings = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

vector_store = []

for chunk in chunks:
    vector_store.append({
        "embedding": embeddings.encode(chunk["text"]),
        "text": chunk["text"],
        "metadata": chunk["metadata"]
        })

In [9]:
vector_store[0]

{'embedding': array([ 7.22921267e-02,  3.64665017e-02,  4.49703000e-02,  8.63403231e-02,
         8.73509794e-02, -6.27046153e-02, -1.02264509e-01, -2.47346181e-02,
        -4.51731645e-02, -8.15360025e-02, -2.54225708e-03, -1.28727898e-01,
         4.06359844e-02,  3.73956077e-02,  6.16961345e-02, -6.56737164e-02,
         2.99233682e-02, -2.18690652e-02, -2.69567892e-02,  2.78616487e-03,
         4.42836732e-02, -1.16460714e-02, -6.49564937e-02,  1.75512359e-02,
        -1.79488515e-03,  5.09238504e-02,  2.83554066e-02, -3.47771868e-02,
        -1.05772680e-03,  2.07425226e-02, -4.69495691e-02, -5.13509847e-02,
         1.63023267e-02,  6.07397929e-02,  3.28542888e-02,  2.19296794e-02,
         8.66114572e-02,  2.78486200e-02, -4.37221862e-02, -7.69941788e-03,
         1.28703204e-03, -2.55825631e-02, -8.49628448e-03,  1.11939432e-02,
         7.30084581e-03,  4.56117615e-02, -1.21031545e-01,  7.09084943e-02,
         5.94530627e-03,  7.04080462e-02, -4.54364270e-02, -5.55896908e-02,

In [10]:
# retrieval
def query_embedding(query_text):
    return embeddings.encode(query_text)

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def retrieve(query_text, top_k):
    results = sorted(
        vector_store,
        key=lambda x: cosine_similarity(query_embedding(query_text), x["embedding"]),
        reverse=True
        )
    
    top_chunks = results[:top_k]

    return top_chunks

In [11]:
query_text = """Symptoms: chest pain, shortness of breath
Lab results: ECG ST elevation, troponin high"""

retrv = retrieve(query_text, 6)

In [12]:
for items in retrv:
    print(items["text"], items["metadata"])

Symptoms: Chest pain radiating to left arm, Shortness of breath, Nausea {'patient_id': 'P_10010', 'chunk_type': 'symptoms', 'department': 'Cardiology'}
Symptoms: Chest tightness for 2 days, Mild shortness of breath, Sweating {'patient_id': 'P_10001', 'chunk_type': 'symptoms', 'department': 'Cardiology'}
Vitals: BP: 170/105, HR: 110
Lab results: ECG: ST elevation, Troponin: Elevated {'patient_id': 'P_10010', 'chunk_type': 'clinical', 'department': 'Cardiology'}
Symptoms: Palpitations, Anxiety, Chest discomfort {'patient_id': 'P_10006', 'chunk_type': 'symptoms', 'department': 'Cardiology'}
Vitals: BP: 140/90, HR: 92
Lab results: ECG: Normal, Troponin: Normal {'patient_id': 'P_10001', 'chunk_type': 'clinical', 'department': 'Cardiology'}
Symptoms: Breathlessness for 1 week, Persistent cough, Wheezing {'patient_id': 'P_10003', 'chunk_type': 'symptoms', 'department': 'Pulmonology'}


In [13]:
patient_groups = defaultdict(list)

for r in retrv:
    patient_groups[r["metadata"]["patient_id"]].append(r)

In [14]:
# getting highest similarity

highest_similarity_patient = ""
highest_similarity_patient_department = ""
c = 0
for pat_id in patient_groups.keys():
    #print(pat_id)
    #print(len(patient_groups[pat_id]))
    #print(patient_groups[pat_id][0]["metadata"]["department"])
    if len(patient_groups[pat_id]) > c:
        highest_similarity_patient, highest_similarity_patient_department = pat_id, patient_groups[pat_id][0]["metadata"]["department"]
        c = len(patient_groups[pat_id])

print("highest_similarity_patient: ", highest_similarity_patient, ", highest_similarity_patient_department: ", highest_similarity_patient_department)

highest_similarity_patient:  P_10010 , highest_similarity_patient_department:  Cardiology


In [15]:
# getting hospital list, doctors and available slots
hospital_list = json.loads(open(os.path.join("Hospital_Details", "hospital_list.json")).read())

selected_hospitals = []
for hospital in hospital_list:
    if highest_similarity_patient_department in hospital["department"]:
        selected_hospitals.append(hospital["hospital"])

selected_hospitals_doctors = []
for hospital in selected_hospitals:
    specific_hospital = {}
    specific_hospital["hospital_name"] = hospital
    file_content = json.loads(open(os.path.join("Hospital_Details", "hospital_" + hospital.lower() + ".json")).read())
    for department in file_content:
        if department["Department"] == highest_similarity_patient_department:
            specific_hospital["doctor_time_slots"] = department["Doctor_List"]
            break
    selected_hospitals_doctors.append(specific_hospital)

selected_hospitals_doctors

[{'hospital_name': 'CCC',
  'doctor_time_slots': [{'Doctor_Name': 'C',
    'Available_Time_Slots': ['02-01-2026 14:00', '02-01-2026 15:00']}]},
 {'hospital_name': 'DDD',
  'doctor_time_slots': [{'Doctor_Name': 'D',
    'Available_Time_Slots': ['02-01-2026 14:00', '02-01-2026 15:00']}]}]